In [1]:
import glob
import os
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [18]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db = db[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
    db['Emotions'] = emotions[i]
    dbs.append(db)
    
db_Train = (pd.concat(dbs[i][:130] for i in range(len(files))))
db_Test = (pd.concat(dbs[i][130:230] for i in range(len(files))))
db = (pd.concat(dbs[i][:] for i in range(len(files))))
X = db_Train[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
Y = db_Train['Emotions']
#X_Norm =(X-X.mean())/X.std()
#X_Cov = np.cov(X_Norm)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
score = clf.score(X,Y)
print(score)

1.0


In [19]:
# Randomly sample 70% of your dataframe
db_0_7 = db_Test.sample(frac=0.7)
# Randomly sample 7 elements from your dataframe
db_7 = db_Test.sample(n=7)
#For either approach above, you can get the rest of the rows by doing:
db_rest = db.loc[~db.index.isin(db_0_7.index)]

In [20]:
db_7

,D1,D2,D3,D4,D5,D6,D7,A0,A1,A2,A3,A4,A5,Emotions
135,16.278821,31.780497,34.481879,7.071068,6.082763,27.294688,57.428216,1.206290,0.348865,0.900236,0.234416,0.902678,1.206290,surprised
186,23.086793,23.194827,24.166092,10.049876,10.000000,25.000000,47.042534,1.302935,0.334156,1.075892,0.360940,0.927295,1.302935,sad
178,31.256999,16.124515,23.853721,7.000000,8.062258,26.019224,81.024688,0.786464,0.280015,1.327977,0.462090,0.565149,0.786464,happy
188,30.000000,30.265492,29.154759,9.000000,9.000000,30.149627,71.253070,1.238293,0.265548,1.015677,0.363653,0.798756,1.238293,neutral
173,28.071338,25.000000,31.384710,13.038405,13.152946,43.011626,55.009090,1.702593,0.209856,0.973620,0.360146,1.281293,1.702593,surprised
178,29.000000,16.124515,21.095023,9.055385,8.000000,17.029386,59.033889,0.942785,0.277694,1.327977,0.408023,0.561291,0.942785,disgusted
141,29.154759,26.172505,30.870698,14.000000,14.035669,33.060551,58.034473,1.499993,0.272957,1.081622,0.340728,1.128422,1.499993,surprised


In [21]:
test = db_7[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
for i in range(len(test)):
    #print(test.iloc[[i]])
    print(clf.predict(test.iloc[[i]]))
    #print(clf.predict_proba(test.iloc[[i]]))
    
   

['fearful']
['neutral']
['happy']
['fearful']
['surprised']
['neutral']
['surprised']


## Generando el arbol

In [22]:
def encode_target(df, target_column):
    """Add column to df with integers for the target.

    Args
    ----
    df -- pandas DataFrame.
    target_column -- column to map to int, producing
                     new Target column.

    Returns
    -------
    df_mod -- modified DataFrame.
    targets -- list of target names.
    """
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)
    

In [23]:
df2, targets = encode_target(db, "Emotions")
print("* df2.head()", df2[["Target", "Emotions"]].head(),
      sep="\n", end="\n\n")
print("* df2.tail()", df2[["Target", "Emotions"]].tail(),
      sep="\n", end="\n\n")
print("* targets", targets, sep="\n", end="\n\n")


* df2.head()
   Target Emotions
0       0  neutral
1       0  neutral
2       0  neutral
3       0  neutral
4       0  neutral

* df2.tail()
     Target   Emotions
225       6  disgusted
226       6  disgusted
227       6  disgusted
228       6  disgusted
229       6  disgusted

* targets
['neutral' 'happy' 'sad' 'fearful' 'angry' 'surprised' 'disgusted']



In [24]:
features = list(df2.columns[:13])
print("* features:", features, sep="\n")
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}
ERtype={0:'neutral', 1: 'happy',
        2: 'sad',3: 'fearful',4:'angry',
        5: 'surprised',6: 'disgusted'}

* features:
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'A0', 'A1', 'A2', 'A3', 'A4', 'A5']


In [25]:
e = ERtype[0]
print(e)

neutral


In [26]:
y = df2["Target"]
X = df2[features]
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, y)
print(db_7['Emotions'])
test = db_7[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
for i in range(len(test)):
    #print(test.iloc[[i]])
    print(dt.predict(test.iloc[[i]]))
    #print(clf.predict_proba(test.iloc[[i]]))

135    surprised
186          sad
178        happy
188      neutral
173    surprised
178    disgusted
141    surprised
Name: Emotions, dtype: object
[3]
[2]
[1]
[5]
[5]
[0]
[5]


In [27]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dotA", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [28]:
visualize_tree(dt, features)
# Aparentemente la grafica no se ve en jupiter pero genera los archivos .dot

## Probando con diferentes clasificadores y verificando su score

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names =  ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process"]
names1 = ["Decision Tree", "Random Forest", "Neural Net", "AdaBoost"]
names2 = ["Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0))]
classifiers1 = [DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=5),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]
classifiers2 = [GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [32]:
emoT = list(db_7['Emotions'])
for name, clf in zip(names2, classifiers2):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        con = 0
        for i in range(len(test)):
            e = clf.predict(test.iloc[[i]])
            if ERtype[e[0]] == emoT[i]:
                print('Comparar: {} con: {}.'.format(ERtype[e[0]],emoT[i]))
                con = con + 1
                      
                      
        print('Porcentaje de acierto', con/len(test)*100)

135    surprised
186          sad
178        happy
188      neutral
173    surprised
178    disgusted
141    surprised
Name: Emotions, dtype: object
Naive Bayes
0.624223602484
Comparar: happy con: happy.
Comparar: surprised con: surprised.
Comparar: disgusted con: disgusted.
Comparar: surprised con: surprised.
Porcentaje de acierto 57.14285714285714
QDA
0.726086956522
Comparar: happy con: happy.
Comparar: surprised con: surprised.
Comparar: surprised con: surprised.
Porcentaje de acierto 42.857142857142854


In [33]:
for name, clf in zip(names1, classifiers1):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        con = 0
        for i in range(len(test)):
            e = clf.predict(test.iloc[[i]])
            if ERtype[e[0]] == emoT[i]:
                print('Comparar: {} con {}:'.format(ERtype[e[0]],emoT[i]))
                con = con + 1
                      
        print('Porcentaje de acierto', con/len(test)*100)

Decision Tree
0.601242236025
Comparar: happy con happy:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 42.857142857142854
Random Forest
0.655900621118
Comparar: happy con happy:
Comparar: neutral con neutral:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 57.14285714285714
Neural Net
0.593167701863
Comparar: happy con happy:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 42.857142857142854
AdaBoost
0.567080745342
Comparar: happy con happy:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 42.857142857142854


In [34]:
emoT = list(db_7['Emotions'])
for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        con = 0
        for i in range(len(test)):
            e = clf.predict(test.iloc[[i]])
            if ERtype[e[0]] == emoT[i]:
                print('Comparar: {} con {}:'.format(ERtype[e[0]],emoT[i]))
                con = con + 1
                
        print('Porcentaje de acierto', con/len(test)*100)

Nearest Neighbors
0.733540372671
Comparar: sad con sad:
Comparar: happy con happy:
Comparar: neutral con neutral:
Comparar: surprised con surprised:
Comparar: disgusted con disgusted:
Comparar: surprised con surprised:
Porcentaje de acierto 85.71428571428571
Linear SVM
0.632919254658
Comparar: happy con happy:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 42.857142857142854
RBF SVM
1.0
Comparar: surprised con surprised:
Comparar: sad con sad:
Comparar: happy con happy:
Comparar: neutral con neutral:
Comparar: surprised con surprised:
Comparar: disgusted con disgusted:
Comparar: surprised con surprised:
Porcentaje de acierto 100.0
Gaussian Process
0.69751552795
Comparar: happy con happy:
Comparar: neutral con neutral:
Comparar: surprised con surprised:
Comparar: surprised con surprised:
Porcentaje de acierto 57.14285714285714


No se está calculando el Gaussian Process?

In [ ]:
print(db_7['Emotions'])